# マルコフモデル

*Markov Model*

マルコフ過程に従う確率モデル。マルコフ連鎖とも。  
マルコフ過程とは、ある状態から次の状態へ遷移する確率が、その状態のみに依存する確率過程のこと。

本章では、このマルコフモデルを用いて言語モデルを実装する。

In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import random
from typing import Dict

import markovify
import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab


---

## マルコフモデルを用いた文章生成

マルコフモデルによる言語モデル（以後マルコフモデルと呼ぶ）では、状態を単語とし、ある単語の次に続く単語の確率を予測する。  

自然な文章を生成するためには、前章で作ったようなランダムな出力を行うモデルではなく、入力された文脈を考慮したモデルが必要となる。  
マルコフモデルはそのようなモデルの中で特に理解しやすいモデルである。このモデルは、与えられた文脈の中の最後の単語に着目し、次の単語を予測する。

マルコフモデルでは、モデルが持っている全ての語彙に対して、その次に続く単語とその確率を記述する。  
例えば、モデルが以下の語彙を持っているとする。

- 今日
- カレー
- 天気
- おいしい
- は
- いい
- 。

これらに対して、次に続く単語とその確率を記述する。  
適当に、主観で決めてみよう。

単語 | 次に続く単語(確率)
--- | ---
今日 | は(1.0)
カレー | は(1.0)
天気 | は(0.5), 。(0.5)
おいしい | 。(0.5), カレー(0.5)
は | 今日(0.25), カレー(0.25), おいしい(0.25), いい(0.25)
いい | 天気(0.5), 。(0.5)

句点は文の終わりを表すので、次に続く単語はないと考える。

マルコフモデルは、この確率に基づいて次の単語を予測する。それを繰り返して文章を生成する。

やってみよう。まずは語彙とその次に続く単語の確率を記述する。

In [2]:
vocab = {
    "今日": {"は": 1},
    "カレー": {"は": 1},
    "天気": {"は": 0.5, "。": 0.5},
    "おいしい": {"。": 0.5, "カレー": 0.5},
    "は": {"今日": 0.25, "カレー": 0.25, "おいしい": 0.25, "いい": 0.25},
    "いい": {"天気": 0.5, "。": 0.5},
}

そして、このような確率に基づいて文章を生成するモデルを作る。

In [3]:
WordProb = Dict[str, float]

class MarkovModel:
    def __init__(self, vocab: Dict[str, WordProb]):
        self.vocab = vocab # 語彙とその次の単語の確率

    def generate_sentence(self, start_word: str) -> str:
        word = start_word
        sentence = [word]

        # 句点が出るまでサンプリング
        while word != "。":
            next_words = self.vocab[word]
            word, = random.choices(list(next_words), weights=next_words.values())
            sentence.append(word)

        return " ".join(sentence)

できた。

初めの単語はこちらで指定する必要がある。  
いくつかの単語で試してみよう。

In [4]:
model = MarkovModel(vocab)
start_words = ["今日", "カレー", "天気"] # 最初の単語
for start_word in start_words:
    sentence = model.generate_sentence(start_word)
    print(sentence)

今日 は 今日 は カレー は 今日 は カレー は おいしい 。
カレー は カレー は いい 天気 は カレー は いい 天気 は 今日 は おいしい 。
天気 。


文脈を考慮したモデルが作成できた。  
次に続く単語に対してまあまあ妥当な確率を手動で設定したので、多少は自然さが見られるはず。


---

## マルコフモデルの学習

語彙と次の単語の確率をデータセットから自動で決定する。

語彙に紐づいている次の単語の確率が、マルコフモデルの全てを表していると見られる。  
前節ではこの確率を手動で定めたが、データを元に自動で決定する事を考える。

このような、モデルの動きを決める値（マルコフモデルの場合は次に続く単語の確率）はパラメータと呼ぶ。そして、適切なパラメータをデータセットから自動で求める事を学習と呼ぶ。

ではやってみよう。

データセットとして、いくつかの単語列を用意する。そして、データセットに出現する全ての単語を対象に、その次に続く単語とその確率を記録する。こうすることでマルコフモデルが完成する。

確率は単語の出現頻度から定める。  
例として以下の3つの単語列をデータセットとして考えてみよう。

- 今日 は いい 天気 です
- 私 は 今日 カレー を 食べ ました
- 私 は カレー が 好き です

これらから適切な確率を定め、マルコフモデルを作成する。

まず、1つ目に選ばれる可能性のある単語として「今日」と「私」がある。どちらが選ばれるかはランダムであるが、「今日」は1回、「私」は2回出現しているので、「今日」が選ばれる確率は1/3、「私」が選ばれる確率は2/3と定義する。  
次に、この次に選ばれる可能性のある単語と確率を定義する。例えば、「今日」の次に来る可能性がある単語は「は」と「カレー」で、出現頻度は同じであるため確率は一様となる。また「私」の後は確定で「は」が来ることになる。  
この流れで全ての単語について次に続く単語と確率を定義するとこうなる。

In [5]:
data = [
    "今日 は いい 天気 です 。",
    "私 は 今日 カレー を 食べ ました 。",
    "私 は カレー が 好き です 。",
]

In [6]:
vocab = {}
for sent in data:
    sent = sent.split()
    for w1, w2 in zip(sent[:-1], sent[1:]):
        if w1 not in vocab:
            vocab[w1] = {}
        if w2 not in vocab[w1]:
            vocab[w1][w2] = 0
        vocab[w1][w2] += 1

for w1 in vocab:
    total = sum(vocab[w1].values())
    for w2 in vocab[w1]:
        vocab[w1][w2] /= total

vocab

{'今日': {'は': 0.5, 'カレー': 0.5},
 'は': {'いい': 0.3333333333333333,
  '今日': 0.3333333333333333,
  'カレー': 0.3333333333333333},
 'いい': {'天気': 1.0},
 '天気': {'です': 1.0},
 'です': {'。': 1.0},
 '私': {'は': 1.0},
 'カレー': {'を': 0.5, 'が': 0.5},
 'を': {'食べ': 1.0},
 '食べ': {'ました': 1.0},
 'ました': {'。': 1.0},
 'が': {'好き': 1.0},
 '好き': {'です': 1.0}}

「今日」の次は「は」と「カレー」が$\frac{1}{2}$ずつ、「は」の次は「いい」「今日」「カレー」が$\frac{1}{3}$ずつという風に確率を定義することが出来た。

これで、マルコフモデルの学習が完了したことになる。  
このモデルでいくつか文章を生成してみよう。

In [7]:
model = MarkovModel(vocab)

for _ in range(10):
    start_word, = random.choices(["今日", "私"], weights=[1/3, 2/3]) # 最初の単語
    sentence = model.generate_sentence(start_word)
    print(sentence)

今日 カレー を 食べ ました 。
私 は 今日 カレー が 好き です 。
私 は 今日 カレー を 食べ ました 。
私 は いい 天気 です 。
今日 は いい 天気 です 。
今日 カレー が 好き です 。
私 は カレー が 好き です 。
今日 は いい 天気 です 。
今日 カレー を 食べ ました 。
今日 カレー が 好き です 。


ランダムなモデルよりは自然な文章が生成されているのではないだろうか。

これで、データセットからマルコフモデルを学習し、文章を生成することが出来た。

### markovify

マルコフモデルを用いて文章を生成するためのライブラリ。

- [jsvine/markovify: A simple, extensible Markov chain generator.](https://github.com/jsvine/markovify)

これを使うと簡単にマルコフモデルを用いた言語モデルを実装できる。

In [8]:
import markovify

In [9]:
data # 再掲

['今日 は いい 天気 です 。', '私 は 今日 カレー を 食べ ました 。', '私 は カレー が 好き です 。']

In [10]:
model = markovify.Text(data, state_size=1) # 学習

これで学習が完了した。文章を生成してみる。

In [11]:
for _ in range(10):
    sentence = model.make_sentence()
    print(sentence)

今日 は カレー を 食べ ました 。
今日 は カレー を 食べ ました 。
今日 は 今日 カレー が 好き です 。
私 は カレー を 食べ ました 。
私 は 今日 カレー が 好き です 。
私 は カレー を 食べ ました 。
私 は 今日 カレー が 好き です 。
私 は カレー を 食べ ました 。
私 は 今日 は 今日 は カレー が 好き です 。
私 は 今日 は カレー が 好き です 。



---

## 一般的なデータセットの活用

ここまで、私が用意した3つの短文をデータセットとして言語モデルを作成した。  
本節ではもう少し大きなデータセットを用いて言語モデルを作成する。

### wiki40b

Wikipediaの記事を集めたデータセット。  
Tensorflow Datasetsに収録されているものを使用する。

[wiki40b  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wiki40b?hl=en#wiki40bja)

In [12]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # tensorflowのログを非表示
import tensorflow as tf
import tensorflow_datasets as tfds

In [13]:
ds = tfds.load("wiki40b/ja", split="test", data_dir="data")
ds = list(ds.as_numpy_iterator())

辞書のリストが得られる。  
`ds: List[Dict[str, bytes]]`

In [14]:
ex = ds[0]
ex

{'text': b'\n_START_ARTICLE_\n\xe3\x83\x93\xe3\x83\xbc\xe3\x83\x88\xe3\x81\x9f\xe3\x81\x91\xe3\x81\x97\xe3\x81\xae\xe6\x95\x99\xe7\xa7\x91\xe6\x9b\xb8\xe3\x81\xab\xe8\xbc\x89\xe3\x82\x89\xe3\x81\xaa\xe3\x81\x84\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xae\xe8\xac\x8e\n_START_SECTION_\n\xe6\xa6\x82\xe8\xa6\x81\n_START_PARAGRAPH_\n\xe3\x80\x8c\xe6\x95\x99\xe7\xa7\x91\xe6\x9b\xb8\xe3\x81\xab\xe3\x81\xaf\xe6\xb1\xba\xe3\x81\x97\xe3\x81\xa6\xe8\xbc\x89\xe3\x82\x89\xe3\x81\xaa\xe3\x81\x84\xe3\x80\x8d\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xae\xe8\xac\x8e\xe3\x82\x84\xe3\x81\x97\xe3\x81\x8d\xe3\x81\x9f\xe3\x82\x8a\xe3\x82\x92\xe5\xa4\x9a\xe8\xa7\x92\xe7\x9a\x84\xe3\x81\xab\xe6\xa4\x9c\xe8\xa8\xbc\xe3\x81\x97\xe3\x80\x81\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xaeDNA\xe3\x82\x92\xe8\xa7\xa3\xe6\x98\x8e\xe3\x81\x99\xe3\x82\x8b\xe3\x80\x82_NEWLINE_\xe6\x96\xb0\xe6\x98\xa5\xe7\x95\xaa\xe7\xb5\x84\xe3\x81\xa8\xe3\x81\x97\xe3\x81\xa6\xe5\xae\x9a\xe6\x9c\x9f\xe7\x9a\x84\xe3\x81\xab\xe6\x

テキストの中身はこんな感じ

In [15]:
print(ex["text"].decode())


_START_ARTICLE_
ビートたけしの教科書に載らない日本人の謎
_START_SECTION_
概要
_START_PARAGRAPH_
「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。


セクションごとにまとめる

In [16]:
data = []
for sample in ds:
    text = sample["text"].decode()
    sections = text.split("_START_SECTION_")
    for section in sections[1:]:
        sentence = section.split("_START_PARAGRAPH_")[1]
        sentence = sentence.replace("_NEWLINE_", "")
        sentence = sentence.replace("\n", "")
        data.append(sentence)

print("num of data:", len(data))
data[:3] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']

後の章でも使うので書き出しておく。

In [17]:
text_path = "data/jawiki.txt"
with open(text_path, "w") as f:
    f.write("\n".join(data))

### トークン化

テキストをトークンごとに分割する。トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。  
本節でも単語を最小単位=トークンとして、文章を単語列に変換する。


英語の文章は単語間にスペースが入っているので、それで終わりである。特にこちらで行う処理はない。  
一方で日本語の文章はそうではないので、こちらで分割する必要がある。

日本語トークン化には形態素解析器を使用する。これは自然言語処理で形態素解析（品詞の分析）に使用するツール。  
色々な種類があるが、ここではMeCabを使用する。

In [18]:
import MeCab

In [19]:
tagger = MeCab.Tagger("-Owakati") # 出力形式を分かち書きに指定
result = tagger.parse("私は猫が好きです。")
result

'私 は 猫 が 好き です 。 \n'

これを使って、学習データの全てを単語ごとに分割する。

In [20]:
data_wakati = []
for sentence in data:
    data_wakati.append(tagger.parse(sentence).strip())

data_wakati[:3] # examples

['「 教科 書 に は 決して 載ら ない 」 日本 人 の 謎 や しきたり を 多角 的 に 検証 し 、 日本 人 の DNA を 解明 する 。 新春 番組 と し て 定期 的 に 放送 さ れ て おり 、 年末 の 午前 中 に 再 放送 さ れる の が 恒例 と なっ て いる 。',
 'ライブ ドア 社員 で あっ た 初代 代表 取締 役 社長 の 山名 真由 に よっ て 企業 内 起業 の 形 で 創業 。 2005 年 に 株式 会社 ライブ ドア から 分割 さ れ て 設立 。 かつて は ライブ ドア ホールディングス （ 現 ・ LDH ） の 子 会社 で あっ た が 、 ノン コア 事業 の 整理 に ともない 、 株式 会社 ゲオ （ 現 ： 株式 会社 ゲオ ホールディングス ） に 所有 する 全 株式 を 譲渡 し 、 同社 の 完全 子 会社 と なっ た 。 「 ぽす れん 」 「 ゲオ 宅配 レンタル 」 の オン ライン DVD ・ CD ・ コミック レンタル サービス 及び 「 GEO Online 」 と 「 ゲオ アプリ 」 の アプリ ・ ウェブサイト 運営 の 大きく 分け て 2 事業 を 展開 し て いる 。 以前 は DVD 販売 等 の E コマース サービス 「 ぽす れん ストア 」 、 動画 配信 コンテンツ 「 ぽす れん BB 」 や 電子 書籍 配信 サービス の 「 GEO ☆ Books 」 事業 も 行っ て い た 。 オン ライン DVD レンタル 事業 で は 会員 数 は 10 万 人 （ 2005 年 9 月 時点 ） 。 2006 年 5 月 より CD レンタル を 開始 。 同業 他社 に は 、 カルチュア ・ コンビニエンス ・ クラブ が 運営 する 『 TSUTAYA DISCAS 』 の ほか 、 DMM . com が 運営 する 『 DMM . com オン ライン DVD レンタル 』 が ある 。 過去 に は 「 Yahoo ! レンタル DVD 」 と 「 楽天 レンタル 」 の 運営 を 受託 し て い た 。',
 '2005 年 の 一 時期 、 東京 の ラジオ 局 、 InterFM で 、 「 堀江 社長 も 使っ

### 学習

markovifyを使ってマルコフモデルを学習させ、文章を生成する。

In [21]:
model = markovify.Text(data_wakati, state_size=1)

In [22]:
for _ in range(5):
    sentence = model.make_sentence().replace(" ", "")
    print(sentence)

沿海地方に位置し独房によりシニアチームが生まれ、国際タイトルを持つことからであったと同様に比例区を調べるようにブラジルで優位性から頻繁な飛行場線に清国である。
1982年7月26日の間部・アランダ川を要する。
大学招聘。
ニトログリセリンのフェアリング装着方法を受け取ることが勃発する。
2016年4得点も、概ね相当するものは、及び西側は新しい者はお告げがある。


語彙が増えたことで多様な文章が生成されるようになった。

ただ、文脈の最後の単語しか考慮できないので、文章全体での自然さを作り出すことは難しい。